In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re

In [2]:
enron = pd.read_csv('./enron_labeled.csv')
enron.head()

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,Cat_10_level_1,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled
0,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


### Combine email bodies and subject lines

In [3]:
enron['Complete'] = None

In [4]:
for i in range(0, enron.shape[0]):
    enron['Complete'][i] = str(str(enron['Subject'][i]) + " " + str(enron['content'][i]))
    
enron.head()

C:\Users\fenes\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled,Complete
0,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Re: Confidential Employee Information/Lenhart ...
1,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...
2,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FW: Western Wholesale Activities - Gas & Power...
3,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FW: Western Wholesale Activities - Gas & Power...
4,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FW: Western Wholesale Activities - Gas & Power...


### Clean
This function will remove certain characters and parts of speech from the text to make clustering more accurate.

The function will return two arrays:
* The cleaned text in its String form
* The cleaned text with its tokenized words and parts-of-speech intact

In [5]:
# nltk's default pos tagger uses the Treebank Tagset.
# This function converts the Treebank tags to the tags used by nltk's WordNetLemmatizer
# In conclusion, this function will allow us to use our pos tags with the lemmatizer.

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN

In [6]:
# Load the nltk stop-words
stop_words = set(stopwords.words("english"))
# Create a lemmatizer
lemmatizer = WordNetLemmatizer()

def clean(text):
    
    cleaned_text_raw = '' # This will hold the cleaned text as a string, without pos tags
    cleaned_text_pos = [] # This will hold the cleaned text as an array of words + their pos tags
    
    # Tokenize the text into sentences
    # Gives an array of strings that make up the message
    lines = sent_tokenize(text)
    
    # For each line in the email, clean the line
    for line in lines:
        
        # Remove the original and forwarded email headers
        # This gets rid of only the headers, which include email addresses and names of employees.
        # The actual "original" and "forwarded" message text remains intact.
        line = re.sub(r'-----Original Message.*Subject: ', '', line)
        line = re.sub(r'-.*Forwarded by.*Subject: ', '', line)
        
        # Remove endline characters
        line = re.sub(' *\r|\n * ', '', line)
        
        # Remove leftover email addresses - 
        # some are found inside email chains, even after the previous line removes forward headers.
        line = re.sub(r'\S*@\S*\s?', '', line)
        
        # When the dataset was released, images were replaced with "<Embedded Picture (Device Independent Bitmap)>"
        # These should be removed as well
        line = re.sub(r'<Embedded Picture .Device Independent Bitmap.>', ' ', line)
        
        # Lastly, some special characters (like '/' and '_') confuse the part-of-speech tagger. Remove these and simply replace them with spaces
        line = re.sub(r'/', '', line)
        line = re.sub(r'\\', '', line)
        line = re.sub(r'_', ' ', line)
        line = re.sub(r'=', ' ', line)
        
        # Tag all the words in the sentence with part of speech
        words = nltk.pos_tag(word_tokenize(line))
             
        # Note: originally, I wanted to remove names because they don't really give context to the
        # email. However, it turns out a lot of things are labelled as proper nouns, so this removes
        # too much context from the email. I want to look into entity recognition to see if I can limit
        # this to only removing peoples' names.
        # words = [w for w in words if not w[1] == "NNP"]
        
        # Remove punctuation (:=: ,=, .=. etc... POS=posessive apostrophes)
        words = [w for w in words if not w[1] in [":", ",", ".", "(", ")", "POS", r"``", r"''"]]
        
        # Convert the words to lowercase
        words = [(w[0].lower(), w[1]) for w in words]
        
        # Lemmatize every word
        words = [(lemmatizer.lemmatize(w[0], get_wordnet_pos(w[1])), w[1]) for w in words] 
        
        # Remove any stopwords from the sentence
        words = [w for w in words if not w[0] in stop_words]
        
        # Remove any numbers from the sentence (CD)
        # words = [w for w in words if not w[1] == "CD"]
        words = [w for w in words if not any(c.isdigit() for c in w[0])]

        
        # Finally, append the cleaned sentence to this list and move on to the next sentence
        if words == []:
            # Ignore lines that cleaning has made empty
            continue
        cleaned_text_pos.append(words)
        
    # Convert the list of tuples to a list of only words
    for line in cleaned_text_pos:
        joined = ' '.join([t[0] for t in line])
        cleaned_text_raw += (str(joined))
        cleaned_text_raw += " "
    
        
    return cleaned_text_raw, cleaned_text_pos

In [7]:
enron['Complete'][15]

'CAISO Notice - Congestion Reform Proposal - Apendix B Market Participants: As part of the Congestion Management Reform Proposal, the ISO has posted Appendix B "Locational Price Dispersion Study" on the ISO web site at . Please be aware that the file is very large (the Word version is 5.1 mb and the PDF version is somewhat smaller at 4.3 mb) so it will take a while to download. Byron Woertz Director, Client Relations'

In [8]:
lines = sent_tokenize(enron['Complete'][14])
cleaned_text = []

for i in range(0, len(lines)):
    words = re.sub('\r|\n ', '', lines[i])
    words = re.sub(r'\\', '', words)
    words = nltk.pos_tag(word_tokenize(words))
    cleaned_text.append(words)
    
# [i[0] for i in cleaned_text[0]]
cleaned_text[len(lines) - 1]

[('Byron', 'NNP'),
 ('Woertz', 'NNP'),
 ('Director', 'NNP'),
 (',', ','),
 ('Client', 'NNP'),
 ('Relations', 'NNP'),
 ('(', '('),
 ('916', 'CD'),
 (')', ')'),
 ('608-7066', 'NN')]

In [9]:
raw, pos = clean(enron['Complete'][13])
print(raw)

[ second delivery wptf friday amen burrito ] sorry gang new computer mess e-mail list 's think fix maybe bear get second time morning gba message-id date fri aug gary ackerman reply-to organization foothill service x-mailer mozilla c-udp ebm-apple macintosh u ppc mime-version webmaster subject wptf friday amen burrito content-type multipartalternative friday burrito fun fortune cookie least accurate everyone get act start gig guy town writing folk like power industry california write new happening important stuff week governor gray davis decide write burrito epistle get press mine muscle turf outdone puc president loretta lynch release report look every facet california power business stone leave unturned tell enough room business u need clear people like herr hair peace governor m-not mr.-rogers davis let lynch need friday burrito re-define reality week mind-numbing aplomb example start early june px order compete business qualified trading vehicle two week later energy f hrer legisla

In [10]:
print(pos)

[[('[', 'JJ'), ('second', 'JJ'), ('delivery', 'NN'), ('wptf', 'NNP'), ('friday', 'NNP'), ('amen', 'NNP'), ('burrito', 'NNP'), (']', 'NNP'), ('sorry', 'NNP'), ('gang', 'NN'), ('new', 'JJ'), ('computer', 'NN'), ('mess', 'VBD'), ('e-mail', 'JJ'), ('list', 'NN')], [("'s", 'VBZ'), ('think', 'VBP'), ('fix', 'VBN')], [('maybe', 'RB')], [('bear', 'NNP'), ('get', 'VBG'), ('second', 'JJ'), ('time', 'NN'), ('morning', 'NN')], [('gba', 'JJ'), ('message-id', 'NN'), ('date', 'NN'), ('fri', 'NNP'), ('aug', 'NNP'), ('gary', 'JJ'), ('ackerman', 'NNP'), ('reply-to', 'NN'), ('organization', 'NN'), ('foothill', 'NNP'), ('service', 'VBZ'), ('x-mailer', 'NN'), ('mozilla', 'NNP'), ('c-udp', 'NNP'), ('ebm-apple', 'JJ'), ('macintosh', 'NNP'), ('u', 'NNP'), ('ppc', 'NNP'), ('mime-version', 'NN'), ('webmaster', 'NN'), ('subject', 'NN'), ('wptf', 'NNP'), ('friday', 'NNP'), ('amen', 'NNP'), ('burrito', 'NNP'), ('content-type', 'NN'), ('multipartalternative', 'NN'), ('friday', 'NNP'), ('burrito', 'NNP'), ('fun', 'N

In [11]:
tags_df = pd.DataFrame(index=range(0, enron.shape[0]), columns=['pos'])

In [12]:
for i in range(0, enron.shape[0]):
    enron['Complete'][i], tags_df['pos'][i] = clean(enron['Complete'][i])
    print(i)

0
1
2


C:\Users\fenes\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [13]:
print(tags_df['pos'][0])
print()
print(enron['Complete'][0])

[[('confidential', 'JJ'), ('employee', 'NNP'), ('informationlenhart', 'NNP'), ('also', 'RB'), ('need', 'VBP'), ('know', 'VB'), ('base', 'JJ'), ('salary', 'NNS'), ('jay', 'NNP'), ('reitmeyer', 'NNP'), ('monique', 'NNP'), ('sanchez', 'NNP')], [('job', 'NN'), ('matt', 'NNP')]]

confidential employee informationlenhart also need know base salary jay reitmeyer monique sanchez job matt 


In [14]:
enron.to_pickle("./enron_cleaned.pkl")
enron.to_csv("./enron_cleaned.csv")

In [15]:
tags_df.to_pickle("./pos_tags.pkl")
tags_df.to_csv("./pos_tags.csv")